In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

covid_cases = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "covid_cases") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

covid_cases.show()

24/10/11 11:20:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/11 11:20:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/11 11:20:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+--------------+----------+
|cases_reported|     dates|
+--------------+----------+
|         20124|2020-01-10|
|         40133|2020-01-15|
|         65005|2020-01-20|
|         30005|2020-02-08|
|         35015|2020-02-19|
|         15015|2020-03-03|
|         35035|2020-03-10|
|         49099|2020-03-14|
|         84045|2020-03-20|
|        100106|2020-03-31|
|         17015|2020-04-04|
|         36035|2020-04-11|
|         50099|2020-04-13|
|         87045|2020-04-22|
|        101101|2020-04-30|
|         40015|2020-05-01|
|         54035|2020-05-09|
|         71099|2020-05-14|
|         82045|2020-05-21|
|         90103|2020-05-25|
+--------------+----------+
only showing top 20 rows



In [2]:
covid_cases.createOrReplaceTempView("covid_cases")

In [3]:
spark.sql('''
with cte as (
select
    month(dates) as monthvalue,
    sum(cases_reported) as total_cases
from
    covid_cases
group by monthvalue),
cte2 as (
select
    *,
    sum(total_cases) over (order by monthvalue) as sm_over
from
    cte)
select
    monthvalue,
    round(ifnull(total_cases/lag(sm_over) over (order by monthvalue),'-')*100,1) as percentage_increase
from
    cte2
''').show()

24/10/11 11:21:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/11 11:21:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/11 11:21:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/11 11:21:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/11 11:21:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/11 11:21:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/11 1

+----------+-------------------+
|monthvalue|percentage_increase|
+----------+-------------------+
|         1|               NULL|
|         2|               51.9|
|         3|              148.9|
|         4|               61.5|
|         5|               57.1|
|         6|               10.2|
|         7|                5.7|
|         8|                3.9|
|         9|                7.9|
|        10|                1.7|
|        11|                6.4|
|        12|                7.4|
+----------+-------------------+

